# Trabajo Practico de Econometria
**Maestrias en Economia y Econometria**

Seed: 1910

In [32]:
import numpy as np
from scipy import stats

# Seed para replicabilidad
SEED = 1910

# Parametros del modelo
N_SIM = 5000
BETA0_TRUE = -3.0
BETA1_TRUE = 0.8
OMEGA_DIAG = np.array([4.0, 9.0, 16.0, 25.0, 36.0])  # varianzas por grupo
N_GROUPS = 5

## Parte 1: Propiedades de muestra finita de FGLS (MCGE)

**Modelo:** $y_i = \beta_0 + \beta_1 x_i + u_i$, con $\beta_0 = -3$, $\beta_1 = 0.8$

$u \sim N(0, \Omega \otimes I_N)$, $\Omega = \text{diag}(4, 9, 16, 25, 36)$, $x \sim U[1, 50]$

In [33]:
def generate_data(n_total, beta0, beta1, rng):
    """
    Genera una muestra de tamano n_total = 5*N del modelo (1).
    Cada grupo j (j=0,...,4) tiene N = n_total/5 observaciones con varianza Omega[j,j].
    """
    N = n_total // N_GROUPS
    x = rng.uniform(1, 50, size=n_total)

    # Errores con heterocedasticidad grupal
    u = np.zeros(n_total)
    for j in range(N_GROUPS):
        sigma_j = np.sqrt(OMEGA_DIAG[j])
        u[j * N:(j + 1) * N] = rng.normal(0, sigma_j, size=N)

    y = beta0 + beta1 * x + u
    return y, x


def ols(y, X):
    """Estimacion por MCC (OLS)."""
    return np.linalg.solve(X.T @ X, X.T @ y)


def fgls(y, X, n_total):
    """
    Estimacion por FGLS (MCGE).
    1. Estimar OLS y obtener residuos
    2. Estimar varianza por grupo con residuos al cuadrado (correccion por g.d.l.)
    3. GLS con Omega_hat estimada
    Retorna coeficientes y errores estandar.
    """
    N = n_total // N_GROUPS
    k = X.shape[1]  # numero de regresores (2: constante + x)

    # Paso 1: OLS
    beta_ols = ols(y, X)
    residuals = y - X @ beta_ols

    # Paso 2: Estimar varianzas por grupo con correccion por grados de libertad
    # Multiplicamos por n/(n-k) para corregir el sesgo de los residuos OLS
    df_correction = n_total / (n_total - k)
    sigma2_hat = np.zeros(n_total)
    for j in range(N_GROUPS):
        idx = slice(j * N, (j + 1) * N)
        sigma2_j = np.mean(residuals[idx] ** 2) * df_correction
        sigma2_hat[idx] = sigma2_j

    # Paso 3: GLS -> beta_hat = (X'Omega_inv X)^{-1} X'Omega_inv y
    Omega_inv = np.diag(1.0 / sigma2_hat)
    XtOiX = X.T @ Omega_inv @ X
    XtOiX_inv = np.linalg.inv(XtOiX)
    beta_fgls = XtOiX_inv @ (X.T @ Omega_inv @ y)

    # Varianza FGLS: (X'Omega_inv X)^{-1}
    se_beta = np.sqrt(np.diag(XtOiX_inv))

    return beta_fgls, se_beta

In [34]:
def gls(y, X, Omega):
    """
    Estimacion por GLS (MCG) con Omega conocida.
    beta_gls = (X' Omega_inv X)^{-1} X' Omega_inv y
    Retorna coeficientes y errores estandar.
    """
    Omega_inv = np.linalg.inv(Omega)
    XtOiX = X.T @ Omega_inv @ X
    XtOiX_inv = np.linalg.inv(XtOiX)
    beta_gls = XtOiX_inv @ (X.T @ Omega_inv @ y)

    # Varianza GLS: (X' Omega_inv X)^{-1}
    se_beta = np.sqrt(np.diag(XtOiX_inv))

    return beta_gls, se_beta

In [35]:
def run_simulation_1a(n_total):
    """
    Simulacion punto 1a para un tamano de muestra dado.
    - Tamano del test: datos generados con beta1=0.8 (H0 verdadera)
    - Poder del test: datos generados con beta1=0 y beta1=0.4 (H0 falsa)
    Usa valores criticos de la normal estandar (distribucion asintotica de FGLS).
    """
    beta0_est = np.zeros(N_SIM)
    beta1_est = np.zeros(N_SIM)
    t_size = np.zeros(N_SIM)
    t_power_0 = np.zeros(N_SIM)
    t_power_04 = np.zeros(N_SIM)

    rng = np.random.RandomState(SEED)

    for sim in range(N_SIM):
        # Datos bajo H0 verdadera: beta1 = 0.8
        y, x = generate_data(n_total, BETA0_TRUE, BETA1_TRUE, rng)
        X = np.column_stack([np.ones(n_total), x])
        beta_hat, se_hat = fgls(y, X, n_total)
        beta0_est[sim] = beta_hat[0]
        beta1_est[sim] = beta_hat[1]
        t_size[sim] = (beta_hat[1] - 0.8) / se_hat[1]

        # Datos bajo beta1 = 0 (poder)
        y0, x0 = generate_data(n_total, BETA0_TRUE, 0.0, rng)
        X0 = np.column_stack([np.ones(n_total), x0])
        b0, se0 = fgls(y0, X0, n_total)
        t_power_0[sim] = (b0[1] - 0.8) / se0[1]

        # Datos bajo beta1 = 0.4 (poder)
        y04, x04 = generate_data(n_total, BETA0_TRUE, 0.4, rng)
        X04 = np.column_stack([np.ones(n_total), x04])
        b04, se04 = fgls(y04, X04, n_total)
        t_power_04[sim] = (b04[1] - 0.8) / se04[1]

    # Valores criticos de la normal estandar (distribucion asintotica)
    cv_1 = stats.norm.ppf(1 - 0.01 / 2)
    cv_5 = stats.norm.ppf(1 - 0.05 / 2)

    results = {
        'n_total': n_total,
        'beta0_mean': np.mean(beta0_est),
        'beta0_median': np.median(beta0_est),
        'beta0_std': np.std(beta0_est),
        'beta1_mean': np.mean(beta1_est),
        'beta1_median': np.median(beta1_est),
        'beta1_std': np.std(beta1_est),
        'size_1': np.mean(np.abs(t_size) > cv_1) * 100,
        'size_5': np.mean(np.abs(t_size) > cv_5) * 100,
        'power_b1_0_at_1': np.mean(np.abs(t_power_0) > cv_1) * 100,
        'power_b1_0_at_5': np.mean(np.abs(t_power_0) > cv_5) * 100,
        'power_b1_04_at_1': np.mean(np.abs(t_power_04) > cv_1) * 100,
        'power_b1_04_at_5': np.mean(np.abs(t_power_04) > cv_5) * 100,
    }
    return results


def print_results(res):
    """Imprime resultados de la simulacion."""
    print("=" * 65)
    print(f"  FGLS - Tamano de muestra: 5N = {res['n_total']}")
    print("=" * 65)

    print()
    print(f"--- Estimaciones de beta0 (verdadero = -3.0) ---")
    print(f"  Media:    {res['beta0_mean']:.6f}")
    print(f"  Mediana:  {res['beta0_median']:.6f}")
    print(f"  Desvio:   {res['beta0_std']:.6f}")

    print()
    print(f"--- Estimaciones de beta1 (verdadero = 0.8) ---")
    print(f"  Media:    {res['beta1_mean']:.6f}")
    print(f"  Mediana:  {res['beta1_median']:.6f}")
    print(f"  Desvio:   {res['beta1_std']:.6f}")

    print()
    print(f"--- Tamano del test (H0: beta1 = 0.8) ---")
    print(f"  Al 1%:  {res['size_1']:.2f}%  (nominal: 1%)")
    print(f"  Al 5%:  {res['size_5']:.2f}%  (nominal: 5%)")

    print()
    print(f"--- Poder del test (H0: beta1 = 0.8) ---")
    print(f"  beta1 = 0:    al 1%: {res['power_b1_0_at_1']:.2f}%  |  al 5%: {res['power_b1_0_at_5']:.2f}%")
    print(f"  beta1 = 0.4:  al 1%: {res['power_b1_04_at_1']:.2f}%  |  al 5%: {res['power_b1_04_at_5']:.2f}%")

### Punto 1a;2-6: 5N = n

In [36]:
for n in [5,10,30,100,200,500]:
    res_5 = run_simulation_1a(n_total=n)
    print_results(res_5)

  FGLS - Tamano de muestra: 5N = 5

--- Estimaciones de beta0 (verdadero = -3.0) ---
  Media:    -2.917793
  Mediana:  -2.934760
  Desvio:   5.597167

--- Estimaciones de beta1 (verdadero = 0.8) ---
  Media:    0.794782
  Mediana:  0.795753
  Desvio:   0.192855

--- Tamano del test (H0: beta1 = 0.8) ---
  Al 1%:  28.64%  (nominal: 1%)
  Al 5%:  38.30%  (nominal: 5%)

--- Poder del test (H0: beta1 = 0.8) ---
  beta1 = 0:    al 1%: 96.96%  |  al 5%: 98.34%
  beta1 = 0.4:  al 1%: 80.82%  |  al 5%: 87.68%
  FGLS - Tamano de muestra: 5N = 10

--- Estimaciones de beta0 (verdadero = -3.0) ---
  Media:    -2.966155
  Mediana:  -3.001572
  Desvio:   3.034392

--- Estimaciones de beta1 (verdadero = 0.8) ---
  Media:    0.799587
  Mediana:  0.799594
  Desvio:   0.105066

--- Tamano del test (H0: beta1 = 0.8) ---
  Al 1%:  16.28%  (nominal: 1%)
  Al 5%:  25.90%  (nominal: 5%)

--- Poder del test (H0: beta1 = 0.8) ---
  beta1 = 0:    al 1%: 99.96%  |  al 5%: 100.00%
  beta1 = 0.4:  al 1%: 95.66%  |

### Punto 1b: Descomposicion de Cholesky

Se busca $P$ tal que $\Omega = P \cdot P'$. Luego se transforma el modelo:

$$P^{-1} y = P^{-1} X \beta + P^{-1} u$$

donde $P^{-1} u$ tiene covarianza $P^{-1} \Omega (P^{-1})' = P^{-1} P P' (P^{-1})' = I$.

Al estimar por OLS el modelo transformado ($y^* = X^* \beta + u^*$) se obtiene el estimador GLS.

In [37]:
def run_simulation_1b(n_total=5):
    """
    Punto 1b: GLS via Cholesky + OLS sobre modelo transformado.
    Usa Omega verdadera (conocida), no estimada.
    Compara con FGLS del punto 1a.
    """
    N = n_total // N_GROUPS

    # Construir Omega completa (5N x 5N): diagonal con varianzas por grupo
    omega_full = np.diag(np.repeat(OMEGA_DIAG, N))

    # Descomposicion de Cholesky: Omega = P * P'
    P = np.linalg.cholesky(omega_full)
    P_inv = np.linalg.inv(P)

    print("Matriz Omega (5x5 para N=1):")
    print(omega_full)
    print("\nMatriz P (Cholesky, Omega = P * P'):")
    print(P)
    print("\nVerificacion P * P' = Omega:", np.allclose(P @ P.T, omega_full))

    # Simulacion
    beta0_gls = np.zeros(N_SIM)
    beta1_gls = np.zeros(N_SIM)
    beta0_fgls = np.zeros(N_SIM)
    beta1_fgls = np.zeros(N_SIM)

    rng = np.random.RandomState(SEED)

    for sim in range(N_SIM):
        y, x = generate_data(n_total, BETA0_TRUE, BETA1_TRUE, rng)
        X = np.column_stack([np.ones(n_total), x])

        # --- GLS via Cholesky: transformar y estimar OLS ---
        y_star = P_inv @ y        # y* = P^{-1} y
        X_star = P_inv @ X        # X* = P^{-1} X
        beta_cholesky = ols(y_star, X_star)
        beta0_gls[sim] = beta_cholesky[0]
        beta1_gls[sim] = beta_cholesky[1]

        # --- FGLS (del punto 1a) para comparar ---
        beta_fgls_hat, _ = fgls(y, X, n_total)
        beta0_fgls[sim] = beta_fgls_hat[0]
        beta1_fgls[sim] = beta_fgls_hat[1]

    # Reportar comparacion
    print("\n" + "=" * 65)
    print(f"  Comparacion GLS (Cholesky+OLS) vs FGLS  |  5N = {n_total}")
    print("=" * 65)

    print("\n--- beta0 (verdadero = -3.0) ---")
    print(f"  {'':20s} {'GLS (Cholesky)':>16s} {'FGLS':>16s}")
    print(f"  {'Media':20s} {np.mean(beta0_gls):16.6f} {np.mean(beta0_fgls):16.6f}")
    print(f"  {'Mediana':20s} {np.median(beta0_gls):16.6f} {np.median(beta0_fgls):16.6f}")
    print(f"  {'Desvio':20s} {np.std(beta0_gls):16.6f} {np.std(beta0_fgls):16.6f}")

    print("\n--- beta1 (verdadero = 0.8) ---")
    print(f"  {'':20s} {'GLS (Cholesky)':>16s} {'FGLS':>16s}")
    print(f"  {'Media':20s} {np.mean(beta1_gls):16.6f} {np.mean(beta1_fgls):16.6f}")
    print(f"  {'Mediana':20s} {np.median(beta1_gls):16.6f} {np.median(beta1_fgls):16.6f}")
    print(f"  {'Desvio':20s} {np.std(beta1_gls):16.6f} {np.std(beta1_fgls):16.6f}")

run_simulation_1b(n_total=5)

Matriz Omega (5x5 para N=1):
[[ 4.  0.  0.  0.  0.]
 [ 0.  9.  0.  0.  0.]
 [ 0.  0. 16.  0.  0.]
 [ 0.  0.  0. 25.  0.]
 [ 0.  0.  0.  0. 36.]]

Matriz P (Cholesky, Omega = P * P'):
[[2. 0. 0. 0. 0.]
 [0. 3. 0. 0. 0.]
 [0. 0. 4. 0. 0.]
 [0. 0. 0. 5. 0.]
 [0. 0. 0. 0. 6.]]

Verificacion P * P' = Omega: True

  Comparacion GLS (Cholesky+OLS) vs FGLS  |  5N = 5

--- beta0 (verdadero = -3.0) ---
                         GLS (Cholesky)             FGLS
  Media                       -3.030895        -3.045046
  Mediana                     -3.066153        -3.024660
  Desvio                       4.850936         5.631405

--- beta1 (verdadero = 0.8) ---
                         GLS (Cholesky)             FGLS
  Media                        0.799237         0.799445
  Mediana                      0.800106         0.798205
  Desvio                       0.166821         0.192225


## Parte 2: Propiedades de muestra finita del test de White

**Modelo:** $y_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \sqrt{\nu_i} u_i$, con $\beta_0 = \beta_1 = \beta_2 = 1$

**Diseno 0:** $u_i \sim N(0,1)$, $\nu_i = 1$ (homocedasticidad)

**Diseno 1:** $u_i \sim N(0,1)$, $\nu_i = e^{0.25 x_{1i} + 0.25 x_{2i}}$ (heterocedasticidad)

**Diseno 2:** $u_i \sim t_5$, $\nu_i = e^{0.25 x_{1i} + 0.25 x_{2i}}$ (no-normalidad + heterocedasticidad)



### 2.1 Test de Heterocedasticidad de White

In [28]:
# ============================================================================
# Generacion de x1 y x2 para la Parte 2
# ============================================================================

def generate_x_part2(n, rng):
    """
    Genera x1 y x2 para el modelo (2).
    Base (n=20): x1 = 18 puntos equiespaciados entre -1 y 1 + extremos -1.1 y 1.1
                 x2 = cuantiles de N(0,1) aleatorios
    Para n > 20: se repiten las 20 observaciones base las veces necesarias.
    """
    # Base de 20 observaciones
    x1_interior = np.linspace(-1, 1, 18)
    x1_base = np.concatenate([[-1.1], x1_interior, [1.1]])  # 20 puntos
    x2_base = rng.normal(0, 1, size=20)

    # Repetir para obtener n observaciones
    reps = n // 20
    x1 = np.tile(x1_base, reps)
    x2 = np.tile(x2_base, reps)

    return x1, x2


def generate_y_part2(x1, x2, design, rng):
    """
    Genera y del modelo (2) segun el diseno.
    y_i = 1 + 1*x1_i + 1*x2_i + sqrt(nu_i)*u_i
    Diseno 0: u~N(0,1), nu=1
    Diseno 1: u~N(0,1), nu=exp(0.25*x1 + 0.25*x2)
    Diseno 2: u~t5,     nu=exp(0.25*x1 + 0.25*x2)
    """
    n = len(x1)

    if design == 0:
        u = rng.normal(0, 1, size=n)
        nu = np.ones(n)
    elif design == 1:
        u = rng.normal(0, 1, size=n)
        nu = np.exp(0.25 * x1 + 0.25 * x2)
    elif design == 2:
        u = rng.standard_t(df=5, size=n)
        nu = np.exp(0.25 * x1 + 0.25 * x2)

    y = 1 + 1 * x1 + 1 * x2 + np.sqrt(nu) * u
    return y


def white_test(y, X):
    """
    Test de White para heterocedasticidad.
    1. OLS del modelo original, obtener residuos
    2. Regresion auxiliar: e^2 ~ 1 + x1 + x2 + x1^2 + x2^2 + x1*x2
    3. Estadistico W = n * R^2 ~ chi2(q)
    Retorna el p-valor del test.
    """
    n = X.shape[0]

    # Paso 1: OLS y residuos
    beta_hat = ols(y, X)
    e = y - X @ beta_hat
    e2 = e ** 2

    # Paso 2: Regresion auxiliar
    # X tiene columnas: [1, x1, x2]
    x1 = X[:, 1]
    x2 = X[:, 2]
    Z = np.column_stack([
        np.ones(n),   # constante
        x1,           # x1
        x2,           # x2
        x1 ** 2,      # x1^2
        x2 ** 2,      # x2^2
        x1 * x2       # x1*x2
    ])

    # OLS de e^2 sobre Z
    alpha_hat = ols(e2, Z)
    e2_fitted = Z @ alpha_hat
    ss_res = np.sum((e2 - e2_fitted) ** 2)
    ss_tot = np.sum((e2 - np.mean(e2)) ** 2)
    R2 = 1 - ss_res / ss_tot

    # Paso 3: Estadistico W = n * R^2
    q = Z.shape[1] - 1  # grados de libertad (regresores sin constante = 5)
    W = n * R2
    p_value = 1 - stats.chi2.cdf(W, df=q)

    return p_value


def run_white_test_simulation(n, designs, rng_seed=SEED):
    """
    Ejecuta 5000 simulaciones del test de White para los disenos dados.
    Reporta tamano (diseno 0) y poder (disenos 1 y 2).
    """
    results = {}

    for design in designs:
        rng = np.random.RandomState(rng_seed)
        # Generar x1, x2 una sola vez (son fijos across simulations)
        rng_x = np.random.RandomState(rng_seed)
        x1, x2 = generate_x_part2(n, rng_x)
        X = np.column_stack([np.ones(n), x1, x2])

        p_values = np.zeros(N_SIM)
        for sim in range(N_SIM):
            y = generate_y_part2(x1, x2, design, rng)
            p_values[sim] = white_test(y, X)

        # Tasa de rechazo a distintos niveles
        reject_1 = np.mean(p_values < 0.01) * 100
        reject_5 = np.mean(p_values < 0.05) * 100
        reject_10 = np.mean(p_values < 0.10) * 100

        results[design] = {
            'reject_1': reject_1,
            'reject_5': reject_5,
            'reject_10': reject_10,
        }

    return results


def print_white_results(results, n):
    """Imprime resultados del test de White."""
    print("=" * 65)
    print(f"  Test de White - n = {n}")
    print("=" * 65)

    if 0 in results:
        r = results[0]
        print(f"\n  Diseno 0 (homocedasticidad) - TAMANO del test:")
        print(f"    Al  1%: {r['reject_1']:.2f}%  (nominal:  1%)")
        print(f"    Al  5%: {r['reject_5']:.2f}%  (nominal:  5%)")
        print(f"    Al 10%: {r['reject_10']:.2f}%  (nominal: 10%)")

    if 1 in results:
        r = results[1]
        print(f"\n  Diseno 1 (normal + heterocedasticidad) - PODER del test:")
        print(f"    Al  1%: {r['reject_1']:.2f}%")
        print(f"    Al  5%: {r['reject_5']:.2f}%")
        print(f"    Al 10%: {r['reject_10']:.2f}%")

    if 2 in results:
        r = results[2]
        print(f"\n  Diseno 2 (t5 + heterocedasticidad) - PODER del test:")
        print(f"    Al  1%: {r['reject_1']:.2f}%")
        print(f"    Al  5%: {r['reject_5']:.2f}%")
        print(f"    Al 10%: {r['reject_10']:.2f}%")

#### 2.1a) Diseno 0 (tamano) + 2.1b) Disenos 1 y 2 (poder) para todos los n

In [29]:
# Correr test de White para todos los tamanos de muestra y los 3 disenos
sample_sizes_p2 = [20, 60, 100, 200, 400, 600]

for n in sample_sizes_p2:
    res = run_white_test_simulation(n, designs=[0, 1, 2])
    print_white_results(res, n)
    print()

  Test de White - n = 20

  Diseno 0 (homocedasticidad) - TAMANO del test:
    Al  1%: 0.48%  (nominal:  1%)
    Al  5%: 4.60%  (nominal:  5%)
    Al 10%: 9.04%  (nominal: 10%)

  Diseno 1 (normal + heterocedasticidad) - PODER del test:
    Al  1%: 0.60%
    Al  5%: 6.00%
    Al 10%: 11.38%

  Diseno 2 (t5 + heterocedasticidad) - PODER del test:
    Al  1%: 0.76%
    Al  5%: 6.92%
    Al 10%: 12.26%

  Test de White - n = 60

  Diseno 0 (homocedasticidad) - TAMANO del test:
    Al  1%: 0.90%  (nominal:  1%)
    Al  5%: 4.86%  (nominal:  5%)
    Al 10%: 9.94%  (nominal: 10%)

  Diseno 1 (normal + heterocedasticidad) - PODER del test:
    Al  1%: 5.14%
    Al  5%: 14.66%
    Al 10%: 23.86%

  Diseno 2 (t5 + heterocedasticidad) - PODER del test:
    Al  1%: 3.44%
    Al  5%: 11.20%
    Al 10%: 18.06%

  Test de White - n = 100

  Diseno 0 (homocedasticidad) - TAMANO del test:
    Al  1%: 1.08%  (nominal:  1%)
    Al  5%: 4.70%  (nominal:  5%)
    Al 10%: 9.78%  (nominal: 10%)

  Diseno 1 

### 2.2 Correccion de la matriz de varianzas y covarianzas (White)

Sesgo relativo: $b_j = \frac{1}{S}\sum_{s=1}^{S} \frac{\widehat{Var}(\hat\beta_j)^{(s)} - Var(\hat\beta_j)}{Var(\hat\beta_j)}$

Sesgo relativo total: $|b_0| + |b_1| + |b_2|$

In [38]:
def white_variance_simulation(n, design, use_true_errors=False, rng_seed=SEED):
    """
    Parte 2.2: Sesgo relativo de la estimacion de White de la matriz de var-cov.
    
    Var_true = (X'X)^{-1} X' Omega_true X (X'X)^{-1}  (varianza verdadera de OLS)
    Var_hat  = (X'X)^{-1} X' Omega_hat  X (X'X)^{-1}  (estimacion de White)
    
    Omega_hat = diag(e_hat^2) si use_true_errors=False (residuos)
    Omega_hat = diag(epsilon^2) si use_true_errors=True (errores verdaderos)
    """
    rng_x = np.random.RandomState(rng_seed)
    x1, x2 = generate_x_part2(n, rng_x)
    X = np.column_stack([np.ones(n), x1, x2])
    
    # Varianza de u segun diseno
    if design == 1:
        var_u = 1.0  # u ~ N(0,1)
    elif design == 2:
        var_u = 5.0 / 3.0  # u ~ t5, Var(t5) = 5/(5-2)
    
    # nu_i para heterocedasticidad
    nu = np.exp(0.25 * x1 + 0.25 * x2)
    
    # Varianza verdadera de epsilon_i = sqrt(nu_i)*u_i es: nu_i * Var(u)
    sigma2_true = nu * var_u
    Omega_true = np.diag(sigma2_true)
    
    # Varianza verdadera de beta_hat OLS: (X'X)^{-1} X' Omega_true X (X'X)^{-1}
    XtX_inv = np.linalg.inv(X.T @ X)
    Var_true = XtX_inv @ X.T @ Omega_true @ X @ XtX_inv
    var_true_diag = np.diag(Var_true)  # [Var(b0), Var(b1), Var(b2)]
    
    # Simulacion
    rng = np.random.RandomState(rng_seed)
    relative_bias = np.zeros((N_SIM, 3))  # para b0, b1, b2
    
    for sim in range(N_SIM):
        # Generar errores
        if design == 1:
            u = rng.normal(0, 1, size=n)
        elif design == 2:
            u = rng.standard_t(df=5, size=n)
        
        epsilon = np.sqrt(nu) * u
        y = 1 + 1 * x1 + 1 * x2 + epsilon
        
        # OLS
        beta_hat = ols(y, X)
        e_hat = y - X @ beta_hat
        
        # Estimacion de White
        if use_true_errors:
            Omega_hat = np.diag(epsilon ** 2)  # errores verdaderos
        else:
            Omega_hat = np.diag(e_hat ** 2)    # residuos OLS
        
        Var_hat = XtX_inv @ X.T @ Omega_hat @ X @ XtX_inv
        var_hat_diag = np.diag(Var_hat)
        
        # Sesgo relativo para esta simulacion
        relative_bias[sim, :] = (var_hat_diag - var_true_diag) / var_true_diag
    
    # Promediar sobre simulaciones
    b = np.mean(relative_bias, axis=0)  # b0, b1, b2
    total_bias = np.sum(np.abs(b))
    
    return b, total_bias


def run_white_variance_all(use_true_errors=False):
    """Corre la simulacion 2.2 para todos los n y disenos."""
    sample_sizes = [20, 60, 100, 200, 400, 600]
    label = "errores verdaderos" if use_true_errors else "residuos OLS"
    
    print("=" * 70)
    print(f"  Sesgo relativo de White ({label})")
    print("=" * 70)
    
    for design in [1, 2]:
        print(f"\n--- Diseno {design} ---")
        print(f"  {'n':>5s}  {'b0':>10s}  {'b1':>10s}  {'b2':>10s}  {'|b0|+|b1|+|b2|':>16s}")
        
        for n in sample_sizes:
            b, total = white_variance_simulation(n, design, use_true_errors)
            print(f"  {n:5d}  {b[0]:10.4f}  {b[1]:10.4f}  {b[2]:10.4f}  {total:16.4f}")


# a-f) Con residuos OLS (estimacion estandar de White)
run_white_variance_all(use_true_errors=False)

  Sesgo relativo de White (residuos OLS)

--- Diseno 1 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20     -0.1592     -0.2099     -0.2123            0.5813
     60     -0.0511     -0.0677     -0.0644            0.1831
    100     -0.0317     -0.0430     -0.0407            0.1154
    200     -0.0160     -0.0204     -0.0204            0.0568
    400     -0.0070     -0.0093     -0.0088            0.0251
    600     -0.0048     -0.0055     -0.0067            0.0171

--- Diseno 2 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20     -0.1546     -0.2071     -0.2226            0.5843
     60     -0.0614     -0.0730     -0.0790            0.2133
    100     -0.0364     -0.0443     -0.0466            0.1273
    200     -0.0182     -0.0204     -0.0250            0.0636
    400     -0.0094     -0.0093     -0.0140            0.0327
    600     -0.0049     -0.0036     -0.0088            0.0173


#### 2.2g) Repetir con errores verdaderos en vez de residuos

In [39]:
# g) Con errores verdaderos (epsilon^2 en vez de e_hat^2)
run_white_variance_all(use_true_errors=True)

  Sesgo relativo de White (errores verdaderos)

--- Diseno 1 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20      0.0018     -0.0008      0.0144            0.0169
     60      0.0006      0.0011      0.0069            0.0086
    100     -0.0006     -0.0015      0.0031            0.0052
    200     -0.0003      0.0004      0.0014            0.0021
    400      0.0008      0.0008      0.0019            0.0034
    600      0.0004      0.0013      0.0005            0.0023

--- Diseno 2 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20     -0.0018     -0.0109     -0.0158            0.0286
     60     -0.0097     -0.0067     -0.0086            0.0250
    100     -0.0050     -0.0039     -0.0036            0.0125
    200     -0.0025     -0.0006     -0.0042            0.0073
    400     -0.0017      0.0005     -0.0035            0.0057
    600      0.0003      0.0030     -0.0019            0.0052
